In [1]:
%load_ext autoreload
%autoreload 2

In [45]:
import sys
sys.path.append("..")

from heritageconnector.entity_matching.filter import Filter
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)


In [46]:
df = pd.read_pickle("../GITIGNORE_DATA/lookup_result.pkl")

df.head(2)

,LINK_ID,PREFERRED_NAME,TITLE_NAME,FIRSTMID_NAME,LASTSUFF_NAME,SUFFIX_NAME,HONORARY_SUFFIX,GENDER,BRIEF_BIO,DESCRIPTION,NOTE,BIRTH_DATE,BIRTH_PLACE,DEATH_DATE,DEATH_PLACE,CAUSE_OF_DEATH,NATIONALITY,OCCUPATION,WEBSITE,AFFILIATION,LINGUISTIC_GROUP,TYPE,REFERENCE_NUMBER,SOURCE,CREATE_DATE,UPDATE_DATE,res_ALL_NOTES,res_WIKIDATA_IDs,res_URLS
0,10243,Brooklyn Arms Company,NaN,NaN,Brooklyn Arms Company,NaN,NaN,N,Y,nan,object record: 1987-1020,c. 1870,"Brooklyn, New York, New York state, United States",NaN,NaN,NaN,American,manufacturer of mathematical instruments,NaN,NaN,NaN,NaN,NaN,N,28-JAN-98,06-NOV-18,nan --- object record: 1987-1020,[],[]
1,10245,"Zenthon, Edward Rupert",NaN,Edward Rupert,Zenthon,NaN,NaN,M,Y,REF: http://www.iwm.org.uk/collections/item/object/1030031461,nan,1920-07,"London, Greater London, England, United Kingdom",c. 2002,NaN,NaN,British,engineer,NaN,NaN,NaN,NaN,NaN,N,28-JAN-98,05-AUG-15,REF: http://www.iwm.org.uk/collections/item/object/1030031461 --- nan,[],[http://www.iwm.org.uk/collections/item/object/1030031461]


In [241]:
df_people = df[df['GENDER'].isin(['M', 'F'])].copy()
df_people_small = df_people.head(600)

In [242]:
f = Filter(df_people_small, "res_WIKIDATA_IDs")

  0%|          | 0/4 [00:00<?, ?it/s]

Getting 176 wikidata documents in pages of 50


100%|██████████| 4/4 [00:06<00:00,  1.52s/it]
../heritageconnector/entity_matching/filter.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [243]:
f.add_property_filter("P31", 'Q5') # human
f.add_label_filter("PREFERRED_NAME", threshold=90, include_aliases=False)

Added filter {'property': ['P31', 'Q5', False]}
Added filter {'label': ['PREFERRED_NAME', 90, False]}


In [244]:
f.view_filters()

Filters: 
 - property: ['P31', 'Q5', False]
 - label: ['PREFERRED_NAME', 90, False]


In [245]:
f.process_dataframe()

/Users/kalyan/.local/share/virtualenvs/heritage-connector-tkmarX41/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
100%|██████████| 161/161 [00:00<00:00, 2063.50it/s]

Processing filter 1 of 2
Processing filter 1 of 2


In [246]:
df_new = f.get_dataframe()

In [247]:
df_new['qcodes_filtered'].value_counts()

[]             490
[Q7029112]       1
[Q250732]        1
[Q2420686]       1
[Q4295]          1
              ... 
[Q955750]        1
[Q7327018]       1
[Q1348145]       1
[Q453972]        1
[Q17309113]      1
Name: qcodes_filtered, Length: 111, dtype: int64

In [208]:
df_new.sort_values('qcodes_filtered', ascending=False)[['PREFERRED_NAME', "DESCRIPTION", "NOTE", "res_WIKIDATA_IDs", "res_URLS", "qcodes_filtered"]]

,PREFERRED_NAME,DESCRIPTION,NOTE,res_WIKIDATA_IDs,res_URLS,qcodes_filtered
17443,"Brande, William Thomas","ODNB: Frank A. J. L. James, ‘Brande, William Thomas (1788–1866)’, Oxford Dictionary of National Biography, Oxford University Press, 2004 [http://www.oxforddnb.com/view/article/3258, accessed 24 April 2014]",nan,[Q999118],[http://www.oxforddnb.com/view/article/3258],[Q999118]
2146,"Nollet, Jean Antoine",http://en.wikipedia.org/wiki/Jean_Antoine_Nollet; http://www.newadvent.org/cathen/11090b.htm,nan,[Q994738],"[http://en.wikipedia.org/wiki/Jean_Antoine_Nollet, http://www.newadvent.org/cathen/11090b.htm]",[Q994738]
7604,"Dolezalek, Friedrich","WIKI: https://de.wikipedia.org/wiki/Friedrich_Dolezalek_(Chemiker)\nVIAF: http://viaf.org/viaf/8170785/#Dolezalek,_Friedrich,_1873-",nan,[Q99227],"[https://de.wikipedia.org/wiki/Friedrich_Dolezalek_(Chemiker), http://viaf.org/viaf/8170785/#Dolezalek]",[Q99227]
16805,"Gilden, Bruce",http://www.brucegilden.com/bio/\n8 Janaury 2014\nhttp://en.wikipedia.org/wiki/Bruce_Gilden\n29 October 2013,American street photographer born 1946; joined Magnum 1998,[Q991646],"[http://www.brucegilden.com/bio/, http://en.wikipedia.org/wiki/Bruce_Gilden]",[Q991646]
3905,"Ereky, Karoly",WIKI: http://en.wikipedia.org/wiki/K%C3%A1roly_Ereky,1919 - devised the term 'biotechnology',[Q990568],[http://en.wikipedia.org/wiki/K%C3%A1roly_Ereky],[Q990568]
...,...,...,...,...,...,...
6328,"Bertillon, Alphonse",http://encyclopedia.jrank.org/Cambridge/entries/030/Alphonse-Bertillon.html,"Alphonse Bertillon was a Parisian policeman. He wanted to improve descriptions of suspected criminals, and hoped a good identification system would quickly identify repeat offenders. He believed that people who were liable to reoffend could be identified through physical characteristics, and that people with certain characteristics were more likely to be criminals.\n\nBertillon was a friend of the surgeon and anthropologist Paul Broca, and borrowed Broca’s ideas about measuring physical differences. In 1881, Bertillon developed techniques and instruments to measure individual features that would not change, for example eye colour, the shape and angles of the ear, brow and nose, and the distances between them. He also developed ways to reliably record other physical data about the body, including identifying marks such as tattoos. He collected vast quantities of data and used the new technology of photography.\n\nBy 1883, Bertillon had made 7336 measurements and identified 49 repeat offenders. By 1884 he had found 241. Bertillon described his system as anthropometry, literally ‘measuring humans’. The system became known as ‘Bertillonage’, and it eventually incorporated body measurements, a verbal description, a photograph and fingerprints. The system relied on progressively sorting records of people with matching characteristics until an individual was identified.\n\nBertillonage was initially popular. However, from the 1890s it was superseded by fingerprinting techniques from India. Frances Galton championed their use in forensic science. Fingerprinting proved far simpler for identifying individuals because each person has a unique fingerprint.",[],[http://encyclopedia.jrank.org/Cambridge/entries/030/Alphonse-Bertillon.html],[]
6327,"Bergmann, Ernst von",WIKI,nan,[],[],[]
6325,"Napoleon, Eugene Louis-Jean Joseph","WIKI: http://en.wikipedia.org/wiki/Napol%C3%A9on,_Prince_Imperial","son of Napoleon III and Eugenie; born Louis Napoleon Eugene John Joseph; His Imperial Highness The Prince Imperial (1856-1870); Louis Napoleon, Prince Imperial of France (1870-1873); Prince Imperial Napoleon, Head of the Imperial house of France (1873-1870); fled to England in 1870, living in Chislehurst, Kent; joined the Royal Artillery, Woolwich; killed during Zulu Wars; the last of the House of Bonaparte",[Q517],[http://en.wikipedia.org/wiki/Napol%C3%A9on],[]
6324,"Uccelli, Filippo",Wellcome Medals coll.,nan,[],[],[]


In [219]:
# number, percentage of people with ID after filtering
len(df_new[df_new['qcodes_filtered'].astype(str) != "[]"]), len(df_new[df_new['qcodes_filtered'].astype(str) != "[]"]) / len(df_new)

(2310, 0.223145285935085)

In [240]:
# number of people with more than one ID after filtering
len(df_new[df_new['qcodes_filtered'].map(lambda d: len(d)) > 1])
df_new[df_new['qcodes_filtered'].map(lambda d: len(d)) > 1]

,LINK_ID,PREFERRED_NAME,TITLE_NAME,FIRSTMID_NAME,LASTSUFF_NAME,SUFFIX_NAME,HONORARY_SUFFIX,GENDER,BRIEF_BIO,DESCRIPTION,NOTE,BIRTH_DATE,BIRTH_PLACE,DEATH_DATE,DEATH_PLACE,CAUSE_OF_DEATH,NATIONALITY,OCCUPATION,WEBSITE,AFFILIATION,LINGUISTIC_GROUP,TYPE,REFERENCE_NUMBER,SOURCE,CREATE_DATE,UPDATE_DATE,res_ALL_NOTES,res_WIKIDATA_IDs,res_URLS,qcodes_filtered
1199,27898,"Clegg, Samuel",NaN,Samuel,Clegg,NaN,NaN,M,Y,"ODNB: G. C. Boase, ‘Clegg, Samuel (1781–1861)’, rev. Trevor I. Williams, Oxford Dictionary of National Biography, Oxford University Press, 2004 [http://www.oxforddnb.com/view/article/5590] Samuel Clegg (1781–1861): doi:10.1093/ref:odnb/5590 Samuel Clegg (1814–1856): doi:10.1093/ref:odnb/5589","Father of Samuel Clegg (1814-1856), civil engineer",1781-03-02,"Manchester, Manchester urban district, Greater Manchester, England, United Kingdom",1861-01-08,"Hampstead, Camden, London, Greater London, England, United Kingdom",illness,British,civil engineer,NaN,NaN,NaN,NaN,NaN,Y,22-MAR-00,08-OCT-11,"ODNB: G. C. Boase, ‘Clegg, Samuel (1781–1861)’, rev. Trevor I. Williams, Oxford Dictionary of National Biography, Oxford University Press, 2004 [http://www.oxforddnb.com/view/article/5590] Samuel Clegg (1781–1861): doi:10.1093/ref:odnb/5590 Samuel Clegg (1814–1856): doi:10.1093/ref:odnb/5589 --- Father of Samuel Clegg (1814-1856), civil engineer","[Q3471083, Q17416358]",[http://www.oxforddnb.com/view/article/5590],"[Q3471083, Q17416358]"
2010,38876,"Radcliffe, John",NaN,John,Radcliffe,NaN,NaN,M,Y,"Robert L. Martensen, ‘Radcliffe, John (bap. 1650, d. 1714)’, Oxford Dictionary of National Biography, Oxford University Press, 2004 [http://www.oxforddnb.com/view/article/22985] John Radcliffe (bap. 1650, d. 1714): doi:10.1093/ref:odnb/22985 John Radcliffe (1690–1729): doi:10.1093/ref:odnb/22986",nan,1650,"Wakefield, Wakefield, West Yorkshire, England, United Kingdom",1714-11-01,"Carshalton, Sutton, Greater London, England, United Kingdom",NaN,English,physician; philanthropist,NaN,NaN,NaN,NaN,NaN,N,13-MAY-02,03-APR-08,"Robert L. Martensen, ‘Radcliffe, John (bap. 1650, d. 1714)’, Oxford Dictionary of National Biography, Oxford University Press, 2004 [http://www.oxforddnb.com/view/article/22985] John Radcliffe (bap. 1650, d. 1714): doi:10.1093/ref:odnb/22985 John Radcliffe (1690–1729): doi:10.1093/ref:odnb/22986 --- nan","[Q922508, Q19351970]",[http://www.oxforddnb.com/view/article/22985],"[Q922508, Q19351970]"
2641,43575,"Penn, John",NaN,John,Penn,NaN,NaN,M,Y,"ODNB: Andrew Lambert, ‘Penn, John (1805–1878)’, Oxford Dictionary of National Biography, Oxford University Press, 2004 [http://www.oxforddnb.com/view/article/21852] John Penn (1805–1878): doi:10.1093/ref:odnb/21852; ODNB: Son of engineer John Penn (1770-1843) - Andrew Lambert, ‘Penn, John (1770–1843)’, Oxford Dictionary of National Biography, Oxford University Press, 2004 [http://www.oxforddnb.com/view/article/21851] John Penn (1770–1843): doi:10.1093/ref:odnb/21851","1826 - following an apprenticeship with his father (John Penn 1770-1843), he constructed the steam gun invented by Jacob Perkins, which was shown to Wellington and other officers, taken to Paris, and later exhibited at the Adelaide Gallery, London. 1826 - elected an associate of the Institution of Civil Engineers. 1843 - approached the Admiralty with an unsolicited offer to install high-power oscillating engines in the Admiralty steam yacht Black Eagle. 1845 - became a member of the Institution of Civil Engineers. 1848 - developed the trunk engine which was subsequently installed in the pioneer screw frigate Arrogant. 1852 - a modified version of the high-power oscillating engine was fitted to the pioneer iron screw steamer the Great Britain. 1853-1856 - served on the Institution of Civil Engineers council. 1854 - in collaboration with Francis Pettit Smith, developed and patented the lignum vitae stern bearing. 1859 - arbitrated the quarrel between Isambard Kingdom Brunel and John Scott Russell over the Great Eastern. 1859 - 

In [231]:
# 100% of people with a wikidata ID we could extract ended up having a link
df[(df['res_WIKIDATA_IDs'].map(lambda d: len(d)) > 0) & (df['qcodes_filtered'].astype(str) == "[]")]

,LINK_ID,PREFERRED_NAME,TITLE_NAME,FIRSTMID_NAME,LASTSUFF_NAME,SUFFIX_NAME,HONORARY_SUFFIX,GENDER,BRIEF_BIO,DESCRIPTION,NOTE,BIRTH_DATE,BIRTH_PLACE,DEATH_DATE,DEATH_PLACE,CAUSE_OF_DEATH,NATIONALITY,OCCUPATION,WEBSITE,AFFILIATION,LINGUISTIC_GROUP,TYPE,REFERENCE_NUMBER,SOURCE,CREATE_DATE,UPDATE_DATE,res_ALL_NOTES,res_WIKIDATA_IDs,res_URLS,qcodes_filtered
